# Ibovespa forecasting using neural networks

## Machine Learning Engineer Nanodegree - Capstone Proposal

### Sagemaker Modelling

- Upload data do S3
- Train Model

### Import python packages

In [1]:
import boto3
import sagemaker

from sagemaker.pytorch import PyTorch

from ibovespa.utils import load_config

In [ ]:
import os
import io
import ast
import boto3
import sagemaker
import numpy as np
import pandas as pd
from ibov.deploy import get_deploy_config, define_model

### Loading Configs

In [2]:
config = load_config()

In [ ]:
dropout = config.get("model").get("dropout")
window = config.get("feature").get("window")
hidden_layer = config.get("model").get("hidden_layer")
lr = config.get("model").get("lr")
seed = config.get("model").get("seed")
epochs = config.get("model").get("epochs")

### Set Sagemaker Session

In [12]:
role = config["sagemaker"]["role"]
region = config["sagemaker"]["region"]
prefix = config["sagemaker"]["bucket_prefx"]

session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

### Upload data to S3

In [13]:
input_data = session.upload_data(path="data/data.csv", bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)
input_scaler = session.upload_data(path="data/scaler.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [18]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibovespa", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge')

In [19]:
estimator.fit({'train': input_data, "config": input_config, "scaler": input_scaler})

2021-01-30 02:37:45 Starting - Starting the training job...
2021-01-30 02:38:08 Starting - Launching requested ML instancesProfilerReport-1611974262: InProgress
......
2021-01-30 02:39:15 Starting - Preparing the instances for training.........
2021-01-30 02:41:04 Downloading - Downloading input data......
2021-01-30 02:42:13 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-30 02:42:16,278 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-30 02:42:16,304 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-30 02:42:16,919 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-30 02:42:17,311 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-01-30 02:42:17,312 sagemaker-containers INFO    

02:42:39, epoch: 8, train: 0.629, valid: 0.044
02:42:43, epoch: 9, train: 0.573, valid: 0.051
02:42:46, epoch: 10, train: 0.573, valid: 0.054
02:42:50, epoch: 11, train: 0.539, valid: 0.041
02:42:55, epoch: 12, train: 0.552, valid: 0.041
02:42:59, epoch: 13, train: 0.529, valid: 0.052
02:43:05, epoch: 14, train: 0.506, valid: 0.045
02:43:10, epoch: 15, train: 0.487, valid: 0.038
02:43:16, epoch: 16, train: 0.48, valid: 0.038
02:43:22, epoch: 17, train: 0.449, valid: 0.059
02:43:28, epoch: 18, train: 0.483, valid: 0.045
02:43:35, epoch: 19, train: 0.454, valid: 0.034
02:43:42, epoch: 20, train: 0.44, valid: 0.033
02:43:49, epoch: 21, train: 0.429, valid: 0.034
02:43:57, epoch: 22, train: 0.421, valid: 0.031
02:44:06, epoch: 23, train: 0.435, valid: 0.029
02:44:14, epoch: 24, train: 0.395, valid: 0.045
02:44:23, epoch: 25, train: 0.411, valid: 0.028
02:44:32, epoch: 26, train: 0.404, valid: 0.026
02:44:41, epoch: 27, train: 0.393, valid: 0.03
02:44:51, epoch: 28, train: 0.364, valid: 0.0

In [ ]:
estimator.latest_training_job.job_name

In [ ]:
deploy_config = get_deploy_config(config)

In [ ]:
def define_model(deploy_config):

    codename = deploy_config.get("codename")
    
    client  = boto3.client("sagemaker")

    model = client.create_model(ModelName=codename,
                                PrimaryContainer=deploy_config.get("primary_container"),
                                ExecutionRoleArn=deploy_config.get("role"))

    return model

In [ ]:
define_model(deploy_config)